In [8]:
from langchain import PromptTemplate, LLMChain
import os
from langchain.vectorstores import Chroma
from getpass import getpass
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
import gradio as gr

### Load Vector Store stored locally

In [38]:
#token=''

In [39]:
model_name = "text-embedding-ada-002"

# get openai api key from platform.openai.com
OPENAI_API_KEY = token

OAIembeddings = OpenAIEmbeddings(
    model=model_name, openai_api_key=OPENAI_API_KEY, disallowed_special=()
)

In [40]:
load_vector_store = Chroma(persist_directory="iupui_store_1", embedding_function=OAIembeddings)
retriever = load_vector_store.as_retriever(search_kwargs={"k":4})

### Prompt Template 

In [41]:
prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [42]:
prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
retriever = load_vector_store.as_retriever(search_kwargs={"k":2})

In [48]:
#Example of retreival
query = "who is the Wiiliam Potter?"
semantic_search = retriever.get_relevant_documents(query)
print(semantic_search)

[Document(page_content='Horn Sheeler, Ph.D.](https://honors.iupui.edu/profiles/kristina-horn-sheeler.html)  [William Potter](https://herron.iupui.edu/about/directory/profiles/potter-william.html)', metadata={'seq_num': 1, 'source': '/Users/abhi/Desktop/IS_unibot/full_data.json', 'url': 'URL: https://crl.iupui.edu/aboutcrl/advisoryboard.html#search'}), Document(page_content='M.P.H.](https://lhsi.iupui.edu/about/about-the-team/brandi-gilbert.html)  [Steven J. Higbee, Ph.D.](https://et.iupui.edu/people/sjhigbee)  [Clayton L. Nicholas](https://et.iupui.edu/people/clnichol)  [Randall J. Roper, Ph.D.](https://science.iupui.edu/biology/people-directory/people/roper-randall.html)  [Keith E. Naugle, Ph.D.](https://shhs.iupui.edu/contact/directory/naugle-keith.html)  [Kristina Horn Sheeler, Ph.D.](https://honors.iupui.edu/profiles/kristina-horn-sheeler.html)  [William', metadata={'seq_num': 1, 'source': '/Users/abhi/Desktop/IS_unibot/full_data.json', 'url': 'URL: https://crl.iupui.edu/aboutcrl/a

### Load the model

In [53]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0.3, model='gpt-3.5-turbo',openai_api_key=token)

In [54]:
chain_type_kwargs = {"prompt": prompt}

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents = True,
    chain_type_kwargs= chain_type_kwargs,
    verbose=True
)


In [55]:
response = qa(query)

print(response)



> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'who is the Wiiliam Potter?', 'result': 'William Potter is a professor at IUPUI.', 'source_documents': [Document(page_content='Horn Sheeler, Ph.D.](https://honors.iupui.edu/profiles/kristina-horn-sheeler.html)  [William Potter](https://herron.iupui.edu/about/directory/profiles/potter-william.html)', metadata={'seq_num': 1, 'source': '/Users/abhi/Desktop/IS_unibot/full_data.json', 'url': 'URL: https://crl.iupui.edu/aboutcrl/advisoryboard.html#search'}), Document(page_content='M.P.H.](https://lhsi.iupui.edu/about/about-the-team/brandi-gilbert.html)  [Steven J. Higbee, Ph.D.](https://et.iupui.edu/people/sjhigbee)  [Clayton L. Nicholas](https://et.iupui.edu/people/clnichol)  [Randall J. Roper, Ph.D.](https://science.iupui.edu/biology/people-directory/people/roper-randall.html)  [Keith E. Naugle, Ph.D.](https://shhs.iupui.edu/contact/directory/naugle-keith.html)  [Kristina Horn Sheeler, Ph.D.](https://honors.iupui.edu/profil